이야기하셨던 세종 데이터 관련해서 여러 방면으로 병합했어요
상황에 맞게 사용하면 될듯합니다.

1. 세종 제외하고 병합 (가장 일반적인듯)
-  기준 연도/월: 예를 들어 2010년 1월 ~ 2022년 12월
- 지역: 세종을 제외한 모든 시도 (서울, 부산 등)
- 장점: 모든 데이터가 빈틈없이 존재 → 모델 안정적
- 단점: 세종 데이터 사용 불가

2. 세종 포함하나, 없는 연도는 NaN 허용
- 세종은 2015년 이전은 결측치(NaN) 로 채움
- 전체는 2010~2022, 세종은 2015~2022만 유효
- 장점: 세종 데이터 활용 가능
- 단점: 모델 학습 시 NaN 처리 필요 (drop, impute 등)
-> 세종도 꼭 포함해야 하거나, NaN 처리 전략을 도입할 수 있을 때 사용

3. 2015~2022로 통일하고 세종 포함
- 세종 기준에 맞춰 모든 시도 데이터도 2015~2022로 잘라냄
- 장점: 전 시도 동일 기간 → 깔끔
- 단점: 다른 지역의 유용한 과거 정보(2010~2014) 버려짐
-> 지역 간 비교가 핵심인 분석일 때 유리

In [114]:
# 데이터 불러오기
import pandas as pd

# 데이터 불러오기 (예시: 파일명/시트명에 맞게 수정)
power = pd.read_excel('filtered_power_data.xlsx')
prod_index = pd.read_excel('trimmed_prod_index_data.xlsx')
temp = pd.read_excel('trimmed_temperature_data.xlsx')
precip = pd.read_excel('trimmed_precipitation_data.xlsx')
export_amount = pd.read_excel('trimmed_export_amount_data.xlsx')
util_rate = pd.read_excel('year_month_util_rate.xlsx')

In [115]:
# 전체 컬럼 확인
dfs = [power, prod_index, temp, precip, export_amount, util_rate]
for i, df in enumerate(dfs):
    print(f"DataFrame {i}:")
    print(df.columns)
    print()

DataFrame 0:
Index(['month', 'region', 'division', 'client_num', 'power_kwh', 'price',
       'mean_price_kwh'],
      dtype='object')

DataFrame 1:
Index(['region', 'month', 'prod_index'], dtype='object')

DataFrame 2:
Index(['month', 'region', 'temp_avg'], dtype='object')

DataFrame 3:
Index(['month', 'region', 'precipitation'], dtype='object')

DataFrame 4:
Index(['month', 'region', 'export_amount'], dtype='object')

DataFrame 5:
Index(['month', 'util_rate', 'prod_cab_index', 'util_rate_index(origin)',
       'util_rate_index(seasonal)'],
      dtype='object')



### 1. 세종 제외, 전체기간: 전국 장기/안정적 예측용

In [116]:
# 세종시 제외, 기간 통일(2010-01 ~ 2022-12)
dfs = [power, prod_index, temp, precip, export_amount]

In [117]:
# 모든 주요 지표 데이터프레임에 자료형 통일
for i in range(len(dfs)):
    # region이 있다면 str로 변환 (혹시 nan이 있으면 'nan'으로 들어가서 병합시 문제가 됨)
    if 'region' in dfs[i].columns:
        dfs[i]['region'] = dfs[i]['region'].astype(str)
    # month를 무조건 datetime으로 변환
    dfs[i]['month'] = pd.to_datetime(dfs[i]['month'])

In [118]:
# power 데이터도 자료형 강제 통일(특히 'region')
power['region'] = power['region'].astype(str)
power['month'] = pd.to_datetime(power['month'])

# 산업용 데이터
power_industry = power[power['division'] == '산업용'][['month', 'region', 'power_kwh']]
power_industry = power_industry.rename(columns={'power_kwh': 'power_kwh_industry'})
# 합계 데이터
power_total = power[power['division'] == '합계'][['month', 'region', 'power_kwh']]
power_total = power_total.rename(columns={'power_kwh': 'power_kwh_total'})

# month, region 기준으로 병합 (outer join로 하면 어느 한쪽만 있는 월/지역도 남음)
power_merged = pd.merge(power_industry, power_total, on=['month', 'region'], how='outer')

dfs[0] = power_merged

In [119]:
# 주요 지표끼리 month, region 기준 inner join (공통 기간/지역만 남김)
from functools import reduce
df_main = reduce(lambda left, right: pd.merge(left, right, on=['month','region'], how='inner'), dfs)

In [120]:
# 전국 단위 보조지표(예: util_rate)는 월 기준으로만 left join
util_rate = pd.read_excel('year_month_util_rate.xlsx')
util_rate['month'] = pd.to_datetime(util_rate['month'])
util_rate_sub = util_rate[['month', 'util_rate']]
df_final = pd.merge(df_main, util_rate_sub, on='month', how='left')

In [139]:
# 결과 확인
print(df_final.head())

       month region power_kwh_industry power_kwh_total  prod_index  temp_avg  \
0 2002-01-01     강원       412,125,715     980,802,845       62.553      -2.4   
1 2002-01-01     경기     2,348,126,225   4,788,402,685       21.171      -0.2   
2 2002-01-01     경남     1,054,021,417   1,771,563,128       72.644       3.0   
3 2002-01-01     경북     1,785,172,159   2,438,066,554       81.953       1.2   
4 2002-01-01     광주       138,516,047     440,809,953       41.767       2.9   

   precipitation  export_amount  util_rate  
0           55.6          22623       75.6  
1           47.6        2217505       75.6  
2           55.8        1341779       75.6  
3           76.6        1310165       75.6  
4           95.7         250587       75.6  


In [123]:
# 파일 저장
df_final.to_csv('병합파일/option1_merged_exclude_sejong.csv', index=False)

In [124]:
# 파일 저장
import pandas as pd

df = pd.read_csv('병합파일/option1_merged_exclude_sejong.csv')
print(df.head())
print(df.columns)
print(df['region'].unique())  # 세종 없는지 확인
print(df.isnull().sum())      # 결측치 확인

        month region power_kwh_industry power_kwh_total  prod_index  temp_avg  \
0  2002-01-01     강원       412,125,715     980,802,845       62.553      -2.4   
1  2002-01-01     경기     2,348,126,225   4,788,402,685       21.171      -0.2   
2  2002-01-01     경남     1,054,021,417   1,771,563,128       72.644       3.0   
3  2002-01-01     경북     1,785,172,159   2,438,066,554       81.953       1.2   
4  2002-01-01     광주       138,516,047     440,809,953       41.767       2.9   

   precipitation  export_amount  util_rate  
0           55.6          22623       75.6  
1           47.6        2217505       75.6  
2           55.8        1341779       75.6  
3           76.6        1310165       75.6  
4           95.7         250587       75.6  
Index(['month', 'region', 'power_kwh_industry', 'power_kwh_total',
       'prod_index', 'temp_avg', 'precipitation', 'export_amount',
       'util_rate'],
      dtype='object')
['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '울산' '인천' '전남' '전북' 

---

### 2. 세종 포함하되, 없는 연도는 NaN 허용 + '전국' 제외 버전
- 전체: 2010~2022, 세종: 2015~2022만 유효
- 장점: 세종 데이터도 사용, 단점: 결측치(NaN) 존재

In [140]:
import pandas as pd
from functools import reduce

# 데이터 불러오기 (파일명/경로 필요시 수정)
power = pd.read_excel('filtered_power_data.xlsx')
prod_index = pd.read_excel('trimmed_prod_index_data.xlsx')
temp = pd.read_excel('trimmed_temperature_data.xlsx')
precip = pd.read_excel('trimmed_precipitation_data.xlsx')
export_amount = pd.read_excel('trimmed_export_amount_data.xlsx')
util_rate = pd.read_excel('year_month_util_rate.xlsx')

In [141]:
# [공통 전처리] month datetime, region 문자열
dfs = [power, prod_index, temp, precip, export_amount]
for i in range(len(dfs)):
    dfs[i]['month'] = pd.to_datetime(dfs[i]['month'])
    dfs[i]['region'] = dfs[i]['region'].astype(str)

In [ ]:
# 산업용 데이터
power_ind = power[power['division']=='산업용'][['month','region','power_kwh']]
power_ind = power_ind.rename(columns={'power_kwh':'power_kwh_industry'})

# 합계 데이터
power_total = power[power['division']=='합계'][['month','region','power_kwh']]
power_total = power_total.rename(columns={'power_kwh':'power_kwh_total'})

#  month, region 기준으로 합치기 (둘 다 컬럼으로 남김)
power_merge = pd.merge(power_ind, power_total, on=['month','region'], how='outer')

In [143]:
dfs[0] = power_merge

In [144]:
# 기준 month/region 조합 전체를 먼저 생성 (2010~2022, 모든 시도+세종 포함)
all_months = pd.date_range('2010-01-01', '2022-12-01', freq='MS')
all_regions = sorted(set().union(*[df['region'].unique() for df in dfs]))

In [145]:
# '전국' 있으면 리스트에서 제외
all_regions = [r for r in all_regions if r != '전국']
full_index = pd.MultiIndex.from_product([all_months, all_regions], names=['month','region'])
df_base = pd.DataFrame(index=full_index).reset_index()

In [146]:
#  각 데이터 left join으로 결측치 NaN 허용하며 병합
df_merged = df_base.copy()
for df in dfs:
    # 필요한 컬럼만 병합 (month, region, 변수)
    value_cols = [col for col in df.columns if col not in ['month','region']]
    df_merged = pd.merge(df_merged, df[['month','region'] + value_cols], on=['month','region'], how='left')

In [147]:
# 전국 단위 보조지표는 month 기준으로만 left join
util_rate['month'] = pd.to_datetime(util_rate['month'])
util_rate_sub = util_rate[['month', 'util_rate']]
df_merged = pd.merge(df_merged, util_rate_sub, on='month', how='left')


In [149]:
# 결과 확인 및 파일 저장
print(df_merged.head(15))  # 세종 2010년 등 결측 확인
df_merged.to_csv('병합파일/option2_merged_include_sejong_nan.csv', index=False)

        month region power_kwh_industry power_kwh_total  prod_index  temp_avg  \
0  2010-01-01     강원       463,950,773   1,480,180,516       76.306      -5.9   
1  2010-01-01     경기     4,018,117,308   8,725,480,264       49.870      -4.5   
2  2010-01-01     경남     1,594,246,998   2,922,774,089      118.283       0.3   
3  2010-01-01     경북     2,544,221,899   3,717,347,647      126.896      -1.4   
4  2010-01-01     광주       228,905,072     712,923,474       75.489       0.5   
5  2010-01-01     대구       535,847,540   1,331,471,928       92.576       0.0   
6  2010-01-01     대전       256,888,382     826,372,244       78.518      -2.7   
7  2010-01-01     부산       703,567,523   1,848,493,050      101.268       3.0   
8  2010-01-01     서울       507,859,467   4,398,185,062      113.099      -4.5   
9  2010-01-01     세종                NaN             NaN         NaN       NaN   
10 2010-01-01     울산     1,899,248,453   2,259,609,470      100.031       1.5   
11 2010-01-01     인천     1,1

In [150]:
# 8. 결과 샘플/확인
print(df_merged.head(15))                # 일부 샘플
print(df_merged[df_merged['region']=='세종'].head(10)) # 세종 데이터 샘플
print(df_merged.isnull().sum())          # 결측치 개수 확인
print(df_merged['region'].unique())      # 모든 시도 포함됐는지 확인

        month region power_kwh_industry power_kwh_total  prod_index  temp_avg  \
0  2010-01-01     강원       463,950,773   1,480,180,516       76.306      -5.9   
1  2010-01-01     경기     4,018,117,308   8,725,480,264       49.870      -4.5   
2  2010-01-01     경남     1,594,246,998   2,922,774,089      118.283       0.3   
3  2010-01-01     경북     2,544,221,899   3,717,347,647      126.896      -1.4   
4  2010-01-01     광주       228,905,072     712,923,474       75.489       0.5   
5  2010-01-01     대구       535,847,540   1,331,471,928       92.576       0.0   
6  2010-01-01     대전       256,888,382     826,372,244       78.518      -2.7   
7  2010-01-01     부산       703,567,523   1,848,493,050      101.268       3.0   
8  2010-01-01     서울       507,859,467   4,398,185,062      113.099      -4.5   
9  2010-01-01     세종                NaN             NaN         NaN       NaN   
10 2010-01-01     울산     1,899,248,453   2,259,609,470      100.031       1.5   
11 2010-01-01     인천     1,1

In [137]:
# 세종시 2010~2015년 실제 데이터 샘플 (대부분 NaN이어야 정상)
print(df_merged[(df_merged['region']=='세종') & (df_merged['month']<'2015-01-01')])

# 세종시 2015년 이후 (값이 차있는지)
print(df_merged[(df_merged['region']=='세종') & (df_merged['month']>='2015-01-01')].head())

# 결측치 비율 간단 체크
print(df_merged.isnull().mean())

          month region power_kwh_industry power_kwh_total  prod_index  \
9    2010-01-01     세종                NaN             NaN         NaN   
27   2010-02-01     세종                NaN             NaN         NaN   
45   2010-03-01     세종                NaN             NaN         NaN   
63   2010-04-01     세종                NaN             NaN         NaN   
81   2010-05-01     세종                NaN             NaN         NaN   
99   2010-06-01     세종                NaN             NaN         NaN   
117  2010-07-01     세종                NaN             NaN         NaN   
135  2010-08-01     세종                NaN             NaN         NaN   
153  2010-09-01     세종                NaN             NaN         NaN   
171  2010-10-01     세종                NaN             NaN         NaN   
189  2010-11-01     세종                NaN             NaN         NaN   
207  2010-12-01     세종                NaN             NaN         NaN   
225  2011-01-01     세종                NaN          

[설명]
- 세종시 포함, 전체 기간(2010~2022) 데이터 틀 유지, 없는 값은 NaN
- 나중에 목적에 따라 결측치 처리(평균 대체, 보간 등) 필요할것 같습니다!
- 세종도 꼭 포함하고 싶을 때 or 결측치 전략 가능할 때 사용하시면 됩니다!
- 전국은 제외 된 버전입니다!

---
#### 혹시 필요할까봐 '전국' 추가된 버전 코드 입니다.

In [204]:
import pandas as pd
from functools import reduce

# 1. 데이터 불러오기
power         = pd.read_excel('filtered_power_data.xlsx')
prod_index    = pd.read_excel('trimmed_prod_index_data.xlsx')
temp          = pd.read_excel('trimmed_temperature_data.xlsx')
precip        = pd.read_excel('trimmed_precipitation_data.xlsx')
export_amount = pd.read_excel('trimmed_export_amount_data.xlsx')
util_rate     = pd.read_excel('year_month_util_rate.xlsx')

# 2. 공통 전처리: month→datetime, region→str
dfs = [power, prod_index, temp, precip, export_amount]
for i in range(len(dfs)):
    dfs[i]['month']  = pd.to_datetime(dfs[i]['month'])
    dfs[i]['region'] = dfs[i]['region'].astype(str)

# 3. 전력: 산업용+합계 둘 다 남기기
power_ind   = power[power['division']=='산업용'][['month','region','power_kwh']].rename(columns={'power_kwh':'power_kwh_industry'})
power_total = power[power['division']=='합계'][['month','region','power_kwh']].rename(columns={'power_kwh':'power_kwh_total'})
power_merge = pd.merge(power_ind, power_total, on=['month','region'], how='outer')
dfs[0] = power_merge

# 4. 전체 month×region 조합 생성 (2010~2022, “전국” 포함)
all_months = pd.date_range('2010-01-01', '2022-12-01', freq='MS')
all_regions = sorted({r for df in dfs for r in df['region'].unique()})
full_index = pd.MultiIndex.from_product([all_months, all_regions], names=['month','region'])
df_base = pd.DataFrame(index=full_index).reset_index()

# 5. left join 으로 NaN 허용 병합
df_merged = df_base.copy()
for df in dfs:
    value_cols = [c for c in df.columns if c not in ['month','region']]
    df_merged = pd.merge(df_merged,
                         df[['month','region'] + value_cols],
                         on=['month','region'],
                         how='left')

# 6. 전국 단위 보조지표(util_rate)도 month 기준 left join
util_rate['month'] = pd.to_datetime(util_rate['month'])
util_sub = util_rate[['month','util_rate']]
df_merged = pd.merge(df_merged, util_sub, on='month', how='left')

# 7. 결과 확인 & 파일 저장 (“+전국” 버전)
print(df_merged.head(15))
print(df_merged['region'].unique())
print(df_merged.isnull().mean())

df_merged.to_csv('병합파일/option2_merged_include_sejong_nan_+전국.csv', index=False)

        month region power_kwh_industry power_kwh_total  prod_index  temp_avg  \
0  2010-01-01     강원       463,950,773   1,480,180,516       76.306      -5.9   
1  2010-01-01     경기     4,018,117,308   8,725,480,264       49.870      -4.5   
2  2010-01-01     경남     1,594,246,998   2,922,774,089      118.283       0.3   
3  2010-01-01     경북     2,544,221,899   3,717,347,647      126.896      -1.4   
4  2010-01-01     광주       228,905,072     712,923,474       75.489       0.5   
5  2010-01-01     대구       535,847,540   1,331,471,928       92.576       0.0   
6  2010-01-01     대전       256,888,382     826,372,244       78.518      -2.7   
7  2010-01-01     부산       703,567,523   1,848,493,050      101.268       3.0   
8  2010-01-01     서울       507,859,467   4,398,185,062      113.099      -4.5   
9  2010-01-01     세종                NaN             NaN         NaN       NaN   
10 2010-01-01     울산     1,899,248,453   2,259,609,470      100.031       1.5   
11 2010-01-01     인천     1,1

### 결측치 없는 버전으로 만들기
Why?
1. 모델 학습(머신러닝/딥러닝 등)에서 결측치가 있으면 에러가 발생하거나 예측력이 떨어질 수 있어서
- 대부분의 ML 알고리즘은 결측치(NaN)를 허용하지 않음
- 결측치가 있으면 ValueError: Input contains NaN 이런 에러 자주 남
2. 분석 결과(통계량, 상관관계, 시각화 등)에 영향이 없도록
- 결측치가 들어가면, 평균, 합계, 상관계수, 시계열 그래프 등 값이 왜곡될 수 있음
- 결측치가 있으면 “자료에 구멍이 뚫린 것”처럼 계산이 안 되는 부분이 생김
3. EDA(탐색적 데이터 분석)나 가설 검정, 시각화 작업을 간편하게 하려고
- 결측치가 없으면, 불필요하게 fillna, interpolate, dropna 등 여러 전처리 코드를 추가로 안 써도 됨

In [ ]:
# 결측치 없는 버전 코드
import pandas as pd

# 1. 데이터 불러오기
# 전국 데이터 없는 버전입니다.
df2 = pd.read_csv('병합파일/option2_merged_include_sejong_nan.csv')

In [152]:
# 결측치가 하나라도 있는 행 전체 삭제
df2_clean = df2.dropna()

In [153]:
print("=== 전체 결측치 삭제 후 ===")
print(df2_clean.head())

=== 전체 결측치 삭제 후 ===
        month region power_kwh_industry power_kwh_total  prod_index  temp_avg  \
0  2010-01-01     강원       463,950,773   1,480,180,516       76.306      -5.9   
1  2010-01-01     경기     4,018,117,308   8,725,480,264       49.870      -4.5   
2  2010-01-01     경남     1,594,246,998   2,922,774,089      118.283       0.3   
3  2010-01-01     경북     2,544,221,899   3,717,347,647      126.896      -1.4   
4  2010-01-01     광주       228,905,072     712,923,474       75.489       0.5   

   precipitation  export_amount  util_rate  
0           33.9        82824.0       79.7  
1           25.4      5975813.0       79.7  
2           23.9      2851941.0       79.7  
3           17.2      3221839.0       79.7  
4           37.1       799712.0       79.7  


In [154]:
print(df2_clean.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2496 entries, 0 to 2651
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   month               2496 non-null   object 
 1   region              2496 non-null   object 
 2   power_kwh_industry  2496 non-null   object 
 3   power_kwh_total     2496 non-null   object 
 4   prod_index          2496 non-null   float64
 5   temp_avg            2496 non-null   float64
 6   precipitation       2496 non-null   float64
 7   export_amount       2496 non-null   float64
 8   util_rate           2496 non-null   float64
dtypes: float64(5), object(4)
memory usage: 195.0+ KB
None


In [155]:
print("shape:", df2_clean.shape)

shape: (2496, 9)


In [157]:
# 특정 변수(prod_index, power_kwh)만 결측치 삭제 (다른 결측치는 남김)
df2_clean2 = df2.dropna(subset=['prod_index', 'power_kwh_industry', 'power_kwh_total'])

In [158]:
print("\n=== prod_index, power_kwh 기준 결측치만 삭제 후 ===")
print(df2_clean2.head())


=== prod_index, power_kwh 기준 결측치만 삭제 후 ===
        month region power_kwh_industry power_kwh_total  prod_index  temp_avg  \
0  2010-01-01     강원       463,950,773   1,480,180,516       76.306      -5.9   
1  2010-01-01     경기     4,018,117,308   8,725,480,264       49.870      -4.5   
2  2010-01-01     경남     1,594,246,998   2,922,774,089      118.283       0.3   
3  2010-01-01     경북     2,544,221,899   3,717,347,647      126.896      -1.4   
4  2010-01-01     광주       228,905,072     712,923,474       75.489       0.5   

   precipitation  export_amount  util_rate  
0           33.9        82824.0       79.7  
1           25.4      5975813.0       79.7  
2           23.9      2851941.0       79.7  
3           17.2      3221839.0       79.7  
4           37.1       799712.0       79.7  


In [159]:
print(df2_clean2.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2496 entries, 0 to 2651
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   month               2496 non-null   object 
 1   region              2496 non-null   object 
 2   power_kwh_industry  2496 non-null   object 
 3   power_kwh_total     2496 non-null   object 
 4   prod_index          2496 non-null   float64
 5   temp_avg            2496 non-null   float64
 6   precipitation       2496 non-null   float64
 7   export_amount       2496 non-null   float64
 8   util_rate           2496 non-null   float64
dtypes: float64(5), object(4)
memory usage: 195.0+ KB
None


In [160]:
print("shape:", df2_clean2.shape)

shape: (2496, 9)


In [161]:
# 결측치가 하나라도 있는 행 전체 삭제 후 저장
df2_clean2.to_csv('병합파일/option2_merged_include_sejong_nan_clean.csv', index=False)

---

### 3. 2015~2022로 전체 기간 통일, 세종 포함
- 세종 기준에 맞춰 모든 시도도 2015~2022년만 사용
- 모든 지역, 모든 월이 깔끔하게 맞춰짐(동일 기간 분석)

In [195]:
import pandas as pd
from functools import reduce

# 1. 데이터 불러오기
power = pd.read_excel('filtered_power_data.xlsx')
prod_index = pd.read_excel('trimmed_prod_index_data.xlsx')
temp = pd.read_excel('trimmed_temperature_data.xlsx')
precip = pd.read_excel('trimmed_precipitation_data.xlsx')
export_amount = pd.read_excel('trimmed_export_amount_data.xlsx')
util_rate = pd.read_excel('year_month_util_rate.xlsx')


In [196]:
# 공통 전처리 (month: datetime, region: str)
for df in [power, prod_index, temp, precip, export_amount]:
    df['month'] = pd.to_datetime(df['month'])
    df['region'] = df['region'].astype(str)

In [197]:
# power는 산업용/합계 둘 다 남기기
power_ind = power[power['division']=='산업용'][['month','region','power_kwh']].rename(columns={'power_kwh':'power_kwh_industry'})
power_tot = power[power['division']=='합계'][['month','region','power_kwh']].rename(columns={'power_kwh':'power_kwh_total'})
power = pd.merge(power_ind, power_tot, on=['month','region'], how='inner')


In [198]:
# 2015~2022년으로 잘라내기
# 세종 데이터가 2015년부터 존재함.
# 기간만 필터링 진행
start, end = '2015-01-01', '2022-12-01'
for i in range(len(dfs)):
    dfs[i]['month'] = pd.to_datetime(dfs[i]['month'])
    dfs[i] = dfs[i][(dfs[i]['month'] >= start) & (dfs[i]['month'] <= end)]

In [199]:
# inner join으로 병합
#  MultiIndex 생성 없이도 모든 DF에 공통으로 존재하는 month+region 조합만 남게됨.
from functools import reduce
df_merged = reduce(
    lambda left, right: pd.merge(left, right, on=['month','region'], how='inner'),
    dfs
)

In [200]:
# 설비가동률은 월 기준만 left join
util_rate['month'] = pd.to_datetime(util_rate['month'])
util_rate_sub = util_rate[['month','util_rate']]
df_merged = pd.merge(df_merged, util_rate_sub, on='month', how='left')

In [201]:
# 전국 단위 보조지표(설비가동률)는 월 기준 left join
util_rate['month'] = pd.to_datetime(util_rate['month'])
util_rate_sub = util_rate[['month', 'util_rate']]
df_merged = pd.merge(df_merged, util_rate_sub, on='month', how='left')


In [202]:
# 확인
print(df_merged.head(15))
print(df_merged['region'].unique())
print(df_merged.isnull().mean())

        month region division  client_num        power_kwh  \
0  2015-01-01     강원      산업용     12,216      585,592,815    
1  2015-01-01     강원       합계    952,308    1,617,029,201    
2  2015-01-01     경기      산업용    113,664    4,997,455,612    
3  2015-01-01     경기       합계  3,999,638   10,014,789,252    
4  2015-01-01     경남      산업용     35,928    1,757,495,823    
5  2015-01-01     경남       합계  1,746,563    3,299,662,507    
6  2015-01-01     경북      산업용     29,475    2,923,805,965    
7  2015-01-01     경북       합계  1,800,409    4,191,097,159    
8  2015-01-01     광주      산업용      5,745      266,092,703    
9  2015-01-01     광주       합계    518,490      776,667,026    
10 2015-01-01     대구      산업용     20,111      602,778,084    
11 2015-01-01     대구       합계    872,131    1,417,382,367    
12 2015-01-01     대전      산업용      5,183      286,480,493    
13 2015-01-01     대전       합계    523,314      875,517,360    
14 2015-01-01     부산      산업용     24,513      723,831,996    

       

In [203]:
# 파일 저장
df_merged.to_csv('병합파일/option3_merged_allregion_2015_2022.csv', index=False)

[ 설명 ]
- 각 DF에서 month/region 형식·타입 통일 후 2015~2022로 필터
- reduce(inner join) 으로 공통 구간·지역만 병합
- 전국 단위 보조지표(util_rate)는 month 기준으로만 left join
- 지역 간 동시비교·최신 트렌드 분석에 최적화